### Gradio inteface to take a data file and out puts a predicted chart

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import  keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime
from keras.models import load_model
import gradio as gr

In [2]:


# Load the Keras model
model = load_model('models/emission.h5')

# Function to preprocess the file
def predict_data(input_file):
    # Load the data from the input file
    df = pd.read_csv(input_file)
    #Drop country from data frame
    df.drop(columns='Country', axis=1,inplace=True)
    
    scaler = StandardScaler()
    scaler.fit(df)
    df_scaled = pd.DataFrame(scaler.transform(df), columns=df.columns)

    # Make predictions using the loaded Keras model
    predictions = model.predict(df_scaled)

    #append predicted values
    df['temp_change'] = predictions

    mean_temp_change =df.groupby('Year')['temp_change'].mean()

    # Plotting the data
    plt.figure(figsize=(10, 8))
    plt.plot(mean_temp_change.index, mean_temp_change.values, marker='o')
    plt.xlabel('Year')
    plt.ylabel('Mean Temperature Change')
    plt.title('Mean Temperature Change by Year')
    plt.grid(True)
    plt.xticks(mean_temp_change.index)
    
    plt.savefig('output_chart.png')

    return 'output_chart.png'


In [3]:
# Create a Gradio interface
iface = gr.Interface(fn=predict_data, 
                     inputs=gr.File(label="Upload Data File"), 
                     outputs="image")

iface.launch()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
